# Kreiranje opisov GUI

## Uvoz knjižnic in nastavitev okolja

In [49]:
import pandas as pd
import os
import base64
from pathlib import Path
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
import csv

load_dotenv()
handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

IMAGE_INPUT_FOLDER = "rico_guis/"
MODEL = "gemini-2.0-flash"
IMAGE_TYPE = "image/jpeg"
TEMP=0.7
OUTPUT_FILE = "rico_guis_descriptions.csv"

## Definicija prompta za generiranje opisa aplikacij

Kot primer sta podana opisa za GUI prikazana na slikah 377.jpg in 1264.jpg


In [51]:
PROMPT = """
Q:
Given a screenshot of a mobile application page, write a clear and concise requirements description as if provided by a client who wants to have an application exactly like the one shown. Focus on functional and design requirements the client would specify, such as the purpose of the app, main features, target users, and expected user experience. Phrase the requirements from the client’s perspective (e.g., 'The application should allow users to...'). Avoid technical implementation details and only describe what the client would expect the app to do and look like.

A:
The application should allow users to search for flights quickly and easily.
Users must be able to select departure and destination cities from input fields.
The app should support both round-trip and one-way flight options.
There must be a date selector for choosing travel dates.
Users should be able to select the travel class (e.g., Economy, Business, First Class).
The system should allow specifying the number of passengers, divided into Adults, Children, and Infants.
The interface should have a prominent "Search Flights" button that starts the search.
The app should remember and display the user’s last searches for quick access.
The design should be clean, intuitive, and optimized for mobile use, with a bright and engaging color scheme.

Q:
Given a screenshot of a mobile application page, write a clear and concise requirements description as if provided by a client who wants to have an application exactly like the one shown. Focus on functional and design requirements the client would specify, such as the purpose of the app, main features, target users, and expected user experience. Phrase the requirements from the client’s perspective (e.g., 'The application should allow users to...'). Avoid technical implementation details and only describe what the client would expect the app to do and look like.

A:
The application should guide users through workout sessions in a simple and engaging way.
The main screen must have a clear "Start Workout" button that begins the training session immediately.
Users should be able to choose the type of workout (e.g., Full Body, Upper Body, Lower Body, etc.).
The app should allow configuration of workout parameters such as number of circuits.
An instructor option should be available, with selectable modes (e.g., Announcer, Silent, or other voice/instruction styles).
The navigation bar at the bottom should give access to key sections: Workout, Learn, Achievements, and Track.
The design should use a clean, minimalistic style with intuitive icons and large touch-friendly buttons.
The app should be optimized for quick use during physical activity, minimizing the need for manual input once a workout begins.

Q:
Given a screenshot of a mobile application page, write a clear and concise requirements description as if provided by a client who wants to have an application exactly like the one shown. Focus on functional and design requirements the client would specify, such as the purpose of the app, main features, target users, and expected user experience. Phrase the requirements from the client’s perspective (e.g., 'The application should allow users to...'). Avoid technical implementation details and only describe what the client would expect the app to do and look like.

A:
"""

## Kodiranje in opisovanje slik mobilnih aplikacij

In [52]:
def encode_image(image_path):
    with open(image_path,"rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


In [53]:
def describe_image(image_path):
    base64_image = encode_image(image_path)

    llm = ChatGoogleGenerativeAI(model=MODEL, temperature=TEMP)

    response = llm.invoke([
        HumanMessage(
            content=[
                {"type": "text", "text": PROMPT},
                {
                    "type": "image_url",
                    "image_url": f"data:{IMAGE_TYPE};base64,{base64_image}"
                }
            ]
        )
    ])

    return response.content

## Pipeline za generiranje opisov in shranjevanje rezultatov

In [54]:
def pipeline():
    directory_path = Path(IMAGE_INPUT_FOLDER)
    data = []

    if not os.path.exists(OUTPUT_FILE):
        with open(OUTPUT_FILE, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["UI_Number", "Summary"])
            writer.writeheader()

    for item in directory_path.iterdir():
        if item.suffix.lower() in [".jpg", ".jpeg"]:
        # if item.name == "7757.jpg":
            try:
                response = describe_image(item)
                summary = response
                new_data = {"UI_Number": item.stem, "Summary": summary}
                data.append(new_data)

                with open(OUTPUT_FILE, mode="a", newline="", encoding="utf-8") as f:
                    writer = csv.DictWriter(f, fieldnames=["UI_Number", "Summary"])
                    writer.writerow(new_data)
            except Exception as e:
                print(f"Napaka pri obdelavi {item.name}: {e}")
                continue

    return pd.DataFrame(data)

In [55]:
df = pipeline()
df

""


### Generiranje posameznih opisov

In [57]:
ids = [3727,8640,12740,14283,20947,27360,27382,27707,31390,32310,33383,34517,35526,37505,40673,43872,43977,44756,47926,53469,54468,56905,58124,59429,59576,61851,64858,65592,67540,68368,69574,70410]
images_path = Path("rico_guis")
summary_path = "dataset.csv"
data = []

if not os.path.exists(summary_path):
    with open(summary_path, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["UI_Number", "Summary"])
        writer.writeheader()

for item in images_path.iterdir():
    if item.suffix.lower() in [".jpg", ".jpeg"] and int(item.name.split(".")[0]) in ids:
        try:
            response = describe_image(item)
            summary = response
            new_data = {"UI_Number": item.stem, "Summary": summary}
            data.append(new_data)

            with open(summary_path, mode="a", newline="", encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=["UI_Number", "Summary"])
                writer.writerow(new_data)
        except Exception as e:
            print(f"Napaka pri obdelavi {item.name}: {e}")
            continue

pd.DataFrame(data)

,UI_Number,Summary
0,8640,"The application should announce events, such a..."
